# 🎯 Lezione 24 — PCA: Principal Component Analysis

## Obiettivi di Apprendimento

| # | Obiettivo | Livello |
|---|-----------|---------|
| 1 | Capire il concetto di riduzione dimensionalità | 🟢 Base |
| 2 | Comprendere varianza spiegata e componenti principali | 🟢 Base |
| 3 | Usare PCA per visualizzazione dati ad alta dimensionalità | 🟡 Intermedio |
| 4 | Scegliere il numero ottimale di componenti | 🟡 Intermedio |
| 5 | Interpretare i loadings delle componenti | 🔴 Avanzato |
| 6 | Applicare PCA come preprocessing per ML | 🔴 Avanzato |

---

## 📚 Indice

1. **Teoria** — Dimensionalità, varianza, autovettori
2. **Schema Mentale** — Workflow PCA
3. **Demo Pratiche** — 5 demo progressive
4. **Esercizi** — 3 esercizi con soluzioni
5. **Conclusione** — Cosa portarsi a casa
6. **Bignami** — Reference card

---

## 🔧 Setup

```python
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_iris, load_digits, make_blobs
```

---

## 📖 1. Teoria

### 1.1 Il Problema della Dimensionalità

Immagina di avere un dataset con **100 feature**. Problemi:

| Problema | Conseguenza |
|----------|-------------|
| **Curse of Dimensionality** | I punti diventano "equidistanti" |
| **Overfitting** | Troppi parametri, pochi campioni |
| **Visualizzazione impossibile** | Non puoi plottare 100 dimensioni |
| **Computazione lenta** | Più feature = più calcoli |

**PCA risolve tutto questo** riducendo le dimensioni mantenendo l'informazione importante!

```
100 feature ────[PCA]────> 2-10 componenti
                              ↓
                        Stessa informazione
                        (o quasi!)
```

### 📊 Intuizione Geometrica

```
PRIMA (2D originale):         DOPO (1D con PCA):
         ●                         
        ●●                    PC1
       ●●●    ← dati         ──●●●●●●●●●●●──
      ●●●●      correlati         ↑
     ●●●●●                    proiezione sulla
                              direzione di max varianza
```

---

### 1.2 Cosa Fa PCA: I 4 Passi

PCA trova le **direzioni di massima varianza** nei dati:

```
ALGORITMO PCA:

1. CENTRA i dati (sottrai la media)
   X_centered = X - mean(X)

2. CALCOLA la matrice di covarianza
   Cov = X_centered.T @ X_centered / (n-1)

3. TROVA autovalori e autovettori
   eigenvalues, eigenvectors = eig(Cov)
   
4. PROIETTA i dati sugli autovettori principali
   X_pca = X_centered @ eigenvectors[:, :k]
```

### 🔑 Concetti Chiave

| Termine | Significato |
|---------|-------------|
| **Componente Principale (PC)** | Direzione di massima varianza |
| **Autovettore** | La direzione della PC |
| **Autovalore** | Quanta varianza spiega quella PC |
| **Varianza Spiegata** | % di informazione catturata |
| **Loadings** | Peso di ogni feature originale nella PC |

---

### 1.3 Varianza Spiegata

La domanda chiave: **quante componenti servono?**

```
Varianza spiegata cumulativa:

PC1: ████████████████████░░░░░░░░░░  60%
PC2: ██████████░░░░░░░░░░░░░░░░░░░░  25%  → Cum: 85%
PC3: ████░░░░░░░░░░░░░░░░░░░░░░░░░░  10%  → Cum: 95%
PC4: ██░░░░░░░░░░░░░░░░░░░░░░░░░░░░   5%  → Cum: 100%

Con solo PC1 + PC2 (2 componenti) mantieni l'85% dell'informazione!
```

### 📐 Regole Pratiche per Scegliere K

| Metodo | Regola |
|--------|--------|
| **Soglia varianza** | Mantieni componenti fino a 90-95% varianza cumulativa |
| **Elbow method** | Cerca il "gomito" nel plot degli autovalori |
| **Kaiser criterion** | Mantieni solo PC con autovalore > 1 (su dati standardizzati) |
| **Scree plot** | Dove la curva diventa "piatta" |

---

### 1.4 Interpretare i Loadings

I **loadings** ti dicono come ogni feature originale contribuisce alla PC:

```
Esempio: PC1 per dati di clienti

Feature          Loading
─────────────────────────
età              +0.45    ← contribuisce molto (positivo)
reddito          +0.52    ← contribuisce molto (positivo)
spesa_mensile    +0.48    ← contribuisce molto (positivo)
n_figli          -0.10    ← contribuisce poco

Interpretazione: PC1 cattura il "potere d'acquisto"
(età + reddito + spesa vanno insieme)
```

### ⚠️ Importante: Scaling!

**PCA è sensibile alla scala!** Feature con valori grandi dominano.

```
SBAGLIATO:                    CORRETTO:
età: 20-80                    StandardScaler prima di PCA!
reddito: 20000-200000  ← domina tutto
```

---

### 1.5 Quando Usare PCA

| ✅ Usa PCA quando... | ❌ Evita PCA quando... |
|----------------------|------------------------|
| Hai troppe feature (>20) | Le feature sono già poche |
| Vuoi visualizzare dati HD | Ogni feature è importante singolarmente |
| C'è multicollinearità | I dati non sono correlati |
| Vuoi ridurre overfitting | Hai bisogno di interpretabilità diretta |
| Come preprocessing per ML | I dati sono categorici |

### 🔄 PCA nel Pipeline ML

```
Dati ─→ StandardScaler ─→ PCA ─→ Modello ML
         (necessario!)    ↓
                      n_components=0.95
                      (mantieni 95% varianza)
```

---

## 🧠 2. Schema Mentale

### Workflow PCA

```
┌─────────────────────────────────────────────────────────────────────┐
│                          PCA WORKFLOW                               │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│  1. PREPARAZIONE                                                    │
│     └── StandardScaler() — OBBLIGATORIO per PCA!                    │
│     └── Verifica: dati numerici, no NaN                             │
│                                                                     │
│  2. PCA ESPLORATIVO (tutte le componenti)                           │
│     └── pca = PCA()  # senza n_components                           │
│     └── pca.fit(X_scaled)                                           │
│     └── Analizza: explained_variance_ratio_                         │
│                                                                     │
│  3. SCEGLI n_components                                             │
│     └── Plot varianza cumulativa                                    │
│     └── Soglia: 90-95% varianza                                     │
│     └── Oppure: PCA(n_components=0.95) automatico                   │
│                                                                     │
│  4. FIT FINALE                                                      │
│     └── pca = PCA(n_components=k)                                   │
│     └── X_pca = pca.fit_transform(X_scaled)                         │
│                                                                     │
│  5. INTERPRETAZIONE                                                 │
│     └── pca.components_ → loadings                                  │
│     └── Quali feature contribuiscono a ogni PC?                     │
│                                                                     │
│  6. USO                                                             │
│     └── Visualizzazione (2D/3D)                                     │
│     └── Input per clustering o classificazione                      │
│     └── Riduzione rumore                                            │
│                                                                     │
└─────────────────────────────────────────────────────────────────────┘
```

### ✅ Checklist Pre-PCA

```
□ Dati scalati con StandardScaler?
□ Nessun valore mancante?
□ Solo feature numeriche?
□ Hai un obiettivo chiaro? (visualizzazione, preprocessing, noise reduction)
```

---

## 🔬 3. Demo Pratiche

### Demo 1 — Primo PCA: Dataset Iris

Visualizziamo il classico dataset Iris (4 feature) in 2D.

In [ ]:
# ============================================
# DEMO 1 — Primo PCA: Dataset Iris
# ============================================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_iris, load_digits

print("="*70)
print("DEMO 1 — PCA sul Dataset Iris")
print("="*70)

# Carica Iris
iris = load_iris()
X_iris = iris.data
y_iris = iris.target
feature_names = iris.feature_names

print(f"📊 Dataset Iris:")
print(f"   - Campioni: {X_iris.shape[0]}")
print(f"   - Feature: {X_iris.shape[1]}")
print(f"   - Feature names: {feature_names}")

# ============================================
# PASSO 1: Scaling (OBBLIGATORIO!)
# ============================================
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_iris)

# ============================================
# PASSO 2: PCA esplorativo
# ============================================
pca_full = PCA()
pca_full.fit(X_scaled)

print("\n📈 Varianza spiegata per componente:")
for i, var in enumerate(pca_full.explained_variance_ratio_):
    cum_var = pca_full.explained_variance_ratio_[:i+1].sum()
    bar = "█" * int(var * 50)
    print(f"   PC{i+1}: {var*100:5.1f}% {bar} (cum: {cum_var*100:.1f}%)")

# ============================================
# PASSO 3: PCA con 2 componenti per visualizzazione
# ============================================
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

print(f"\n🎯 Con 2 componenti manteniamo: {pca.explained_variance_ratio_.sum()*100:.1f}% della varianza")

# ============================================
# PASSO 4: Visualizzazione
# ============================================
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot PCA
colors = ['red', 'green', 'blue']
target_names = iris.target_names

for i, (color, name) in enumerate(zip(colors, target_names)):
    mask = y_iris == i
    axes[0].scatter(X_pca[mask, 0], X_pca[mask, 1], c=color, label=name, 
                    s=50, alpha=0.7, edgecolors='black')

axes[0].set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]*100:.1f}%)', fontsize=11)
axes[0].set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]*100:.1f}%)', fontsize=11)
axes[0].set_title('Iris Dataset in 2D (PCA)', fontsize=12)
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Scree plot
axes[1].bar(range(1, 5), pca_full.explained_variance_ratio_, alpha=0.7, 
            label='Varianza individuale', color='steelblue')
axes[1].plot(range(1, 5), np.cumsum(pca_full.explained_variance_ratio_), 
             'ro-', label='Varianza cumulativa')
axes[1].axhline(y=0.95, color='red', linestyle='--', alpha=0.5, label='Soglia 95%')
axes[1].set_xlabel('Componente Principale', fontsize=11)
axes[1].set_ylabel('Varianza Spiegata', fontsize=11)
axes[1].set_title('Scree Plot', fontsize=12)
axes[1].legend()
axes[1].set_xticks(range(1, 5))
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("""
🎯 OSSERVAZIONI:
   1. Con solo 2 PC manteniamo il 95%+ della varianza
   2. Le 3 specie sono chiaramente separabili in 2D
   3. PC1 cattura la maggior parte della varianza (~73%)
""")

---

### Demo 2 — Interpretare i Loadings

Vediamo cosa significano le componenti principali in termini di feature originali.

In [ ]:
# ============================================
# DEMO 2 — Interpretare i Loadings
# ============================================

print("="*70)
print("DEMO 2 — Interpretare i Loadings")
print("="*70)

# I loadings sono in pca.components_
# Ogni riga è una PC, ogni colonna è una feature originale

loadings = pca.components_
df_loadings = pd.DataFrame(
    loadings,
    columns=feature_names,
    index=['PC1', 'PC2']
)

print("\n📊 Loadings (peso di ogni feature nelle PC):")
print(df_loadings.round(3))

# Visualizzazione
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Heatmap dei loadings
im = axes[0].imshow(loadings, cmap='RdBu_r', aspect='auto', vmin=-1, vmax=1)
axes[0].set_xticks(range(len(feature_names)))
axes[0].set_xticklabels([f.replace(' (cm)', '') for f in feature_names], rotation=45, ha='right')
axes[0].set_yticks(range(2))
axes[0].set_yticklabels(['PC1', 'PC2'])
axes[0].set_title('Heatmap dei Loadings', fontsize=12)
plt.colorbar(im, ax=axes[0], label='Loading')

# Aggiungi valori
for i in range(2):
    for j in range(4):
        axes[0].text(j, i, f'{loadings[i, j]:.2f}', ha='center', va='center', 
                     color='white' if abs(loadings[i, j]) > 0.5 else 'black')

# Biplot
ax = axes[1]
# Scatter dei punti
for i, (color, name) in enumerate(zip(colors, target_names)):
    mask = y_iris == i
    ax.scatter(X_pca[mask, 0], X_pca[mask, 1], c=color, label=name, 
               s=30, alpha=0.5)

# Aggiungi le frecce per i loadings
scale = 3  # Scala per visibilità
for i, feature in enumerate(feature_names):
    ax.arrow(0, 0, loadings[0, i]*scale, loadings[1, i]*scale,
             head_width=0.1, head_length=0.1, fc='black', ec='black')
    ax.text(loadings[0, i]*scale*1.15, loadings[1, i]*scale*1.15, 
            feature.replace(' (cm)', ''), fontsize=9, ha='center')

ax.set_xlabel('PC1', fontsize=11)
ax.set_ylabel('PC2', fontsize=11)
ax.set_title('Biplot: Dati + Loadings', fontsize=12)
ax.legend(loc='lower left')
ax.grid(True, alpha=0.3)
ax.axhline(y=0, color='gray', linestyle='-', linewidth=0.5)
ax.axvline(x=0, color='gray', linestyle='-', linewidth=0.5)

plt.tight_layout()
plt.show()

print("""
📌 INTERPRETAZIONE:

PC1 (73% varianza):
   - Petal length (+0.58) e petal width (+0.57) dominano
   - Separa fiori con petali grandi da petali piccoli
   → "Dimensione del petalo"

PC2 (23% varianza):
   - Sepal width (+0.60) domina
   - Distingue fiori con sepali larghi vs stretti
   → "Larghezza del sepalo"
""")

---

### Demo 3 — Scegliere n_components Automaticamente

sklearn permette di specificare la varianza target invece del numero di componenti.

In [ ]:
# ============================================
# DEMO 3 — Scegliere n_components Automaticamente
# ============================================

print("="*70)
print("DEMO 3 — Selezione Automatica di n_components")
print("="*70)

# Dataset più grande: digits (64 feature = 8x8 pixel)
digits = load_digits()
X_digits = digits.data
y_digits = digits.target

print(f"📊 Dataset Digits:")
print(f"   - Campioni: {X_digits.shape[0]}")
print(f"   - Feature: {X_digits.shape[1]} (immagini 8x8)")

# Scaling
scaler = StandardScaler()
X_digits_scaled = scaler.fit_transform(X_digits)

# ============================================
# Metodo 1: Specificare la varianza target
# ============================================
print("\n" + "="*50)
print("METODO 1: Varianza Target")
print("="*50)

soglie = [0.80, 0.90, 0.95, 0.99]

for soglia in soglie:
    pca = PCA(n_components=soglia)
    pca.fit(X_digits_scaled)
    print(f"   Varianza {soglia*100:.0f}% → {pca.n_components_} componenti")

# ============================================
# Metodo 2: Scree Plot + Gomito
# ============================================
print("\n" + "="*50)
print("METODO 2: Scree Plot")
print("="*50)

pca_full = PCA()
pca_full.fit(X_digits_scaled)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Scree plot
axes[0].plot(range(1, 65), pca_full.explained_variance_ratio_, 'b-o', markersize=3)
axes[0].set_xlabel('Componente', fontsize=11)
axes[0].set_ylabel('Varianza Spiegata', fontsize=11)
axes[0].set_title('Scree Plot - Varianza per Componente', fontsize=12)
axes[0].grid(True, alpha=0.3)
axes[0].set_xlim(0, 65)

# Varianza cumulativa
cumsum = np.cumsum(pca_full.explained_variance_ratio_)
axes[1].plot(range(1, 65), cumsum, 'g-o', markersize=3)
axes[1].axhline(y=0.90, color='orange', linestyle='--', label='90%')
axes[1].axhline(y=0.95, color='red', linestyle='--', label='95%')
axes[1].set_xlabel('Componente', fontsize=11)
axes[1].set_ylabel('Varianza Cumulativa', fontsize=11)
axes[1].set_title('Varianza Cumulativa', fontsize=12)
axes[1].legend()
axes[1].grid(True, alpha=0.3)
axes[1].set_xlim(0, 65)

# Trova dove raggiungiamo 90% e 95%
n_90 = np.argmax(cumsum >= 0.90) + 1
n_95 = np.argmax(cumsum >= 0.95) + 1
axes[1].axvline(x=n_90, color='orange', linestyle=':', alpha=0.7)
axes[1].axvline(x=n_95, color='red', linestyle=':', alpha=0.7)

plt.tight_layout()
plt.show()

print(f"""
📊 RISULTATI:
   - Per 90% varianza: {n_90} componenti (da 64!)
   - Per 95% varianza: {n_95} componenti
   
   Riduzione: 64 → {n_90} = {100*(1-n_90/64):.0f}% meno dimensioni!
""")

---

### Demo 4 — Visualizzazione di Dati ad Alta Dimensionalità

Visualizziamo le cifre scritte a mano (64D) in 2D.

In [ ]:
# ============================================
# DEMO 4 — Visualizzazione Dati Alta Dimensionalità
# ============================================

print("="*70)
print("DEMO 4 — Visualizzazione Digits in 2D e 3D")
print("="*70)

# PCA a 2 componenti
pca_2d = PCA(n_components=2)
X_digits_2d = pca_2d.fit_transform(X_digits_scaled)

# PCA a 3 componenti
pca_3d = PCA(n_components=3)
X_digits_3d = pca_3d.fit_transform(X_digits_scaled)

print(f"📊 Varianza spiegata:")
print(f"   2D: {pca_2d.explained_variance_ratio_.sum()*100:.1f}%")
print(f"   3D: {pca_3d.explained_variance_ratio_.sum()*100:.1f}%")

fig = plt.figure(figsize=(16, 5))

# Alcune immagini originali
ax1 = fig.add_subplot(131)
for i in range(10):
    ax1.imshow(digits.images[i], cmap='gray', extent=[i*9, i*9+8, 0, 8])
ax1.set_xlim(0, 90)
ax1.set_ylim(-1, 9)
ax1.set_title('Esempi di Cifre (8x8 pixel)', fontsize=12)
ax1.axis('off')

# 2D scatter
ax2 = fig.add_subplot(132)
scatter = ax2.scatter(X_digits_2d[:, 0], X_digits_2d[:, 1], 
                       c=y_digits, cmap='tab10', s=10, alpha=0.6)
ax2.set_xlabel(f'PC1 ({pca_2d.explained_variance_ratio_[0]*100:.1f}%)')
ax2.set_ylabel(f'PC2 ({pca_2d.explained_variance_ratio_[1]*100:.1f}%)')
ax2.set_title('Digits in 2D (PCA)', fontsize=12)
plt.colorbar(scatter, ax=ax2, label='Cifra')

# 3D scatter
ax3 = fig.add_subplot(133, projection='3d')
scatter3d = ax3.scatter(X_digits_3d[:, 0], X_digits_3d[:, 1], X_digits_3d[:, 2],
                         c=y_digits, cmap='tab10', s=10, alpha=0.6)
ax3.set_xlabel('PC1')
ax3.set_ylabel('PC2')
ax3.set_zlabel('PC3')
ax3.set_title('Digits in 3D (PCA)', fontsize=12)

plt.tight_layout()
plt.show()

# Zoom su alcune cifre
fig, axes = plt.subplots(2, 5, figsize=(15, 6))

for digit in range(10):
    ax = axes[digit // 5, digit % 5]
    mask = y_digits == digit
    ax.scatter(X_digits_2d[~mask, 0], X_digits_2d[~mask, 1], c='lightgray', s=5, alpha=0.3)
    ax.scatter(X_digits_2d[mask, 0], X_digits_2d[mask, 1], c='red', s=10, alpha=0.7)
    ax.set_title(f'Cifra {digit}', fontsize=11)
    ax.set_xticks([])
    ax.set_yticks([])

plt.suptitle('Distribuzione di ogni cifra nello spazio PCA 2D', fontsize=13)
plt.tight_layout()
plt.show()

print("""
🎯 OSSERVAZIONI:
   1. Cifre simili (es. 3,5,8) sono vicine nello spazio PCA
   2. Cifre diverse (es. 0,1) sono ben separate
   3. Con solo 2 PC (~30% varianza) vediamo già la struttura!
""")

---

### Demo 5 — PCA per Riduzione Rumore

PCA può essere usato per rimuovere il rumore ricostruendo i dati con meno componenti.

In [ ]:
# ============================================
# DEMO 5 — PCA per Riduzione Rumore
# ============================================

print("="*70)
print("DEMO 5 — PCA per Riduzione Rumore")
print("="*70)

# Prendi alcune cifre e aggiungi rumore
np.random.seed(42)
indices = [0, 100, 200, 300, 400]  # 5 cifre diverse
X_sample = digits.images[indices].copy()

# Aggiungi rumore gaussiano
noise_level = 5
X_noisy = X_sample + np.random.normal(0, noise_level, X_sample.shape)
X_noisy = np.clip(X_noisy, 0, 16)  # Clip ai valori validi

# Flatten per PCA
X_noisy_flat = X_noisy.reshape(5, 64)

# Ricostruzione con diversi numeri di componenti
n_components_list = [5, 10, 20, 40]

fig, axes = plt.subplots(len(indices), len(n_components_list) + 2, figsize=(16, 10))

for i, idx in enumerate(indices):
    # Originale
    axes[i, 0].imshow(X_sample[i], cmap='gray')
    if i == 0:
        axes[i, 0].set_title('Originale', fontsize=10)
    axes[i, 0].axis('off')
    
    # Con rumore
    axes[i, 1].imshow(X_noisy[i], cmap='gray')
    if i == 0:
        axes[i, 1].set_title(f'+ Rumore\n(σ={noise_level})', fontsize=10)
    axes[i, 1].axis('off')
    
    # Ricostruzioni
    for j, n_comp in enumerate(n_components_list):
        # Fit PCA su tutto il dataset digits per avere una buona base
        pca_denoise = PCA(n_components=n_comp)
        pca_denoise.fit(X_digits_scaled)
        
        # Trasforma e ricostruisci la singola immagine rumorosa
        x_noisy_scaled = (X_noisy_flat[i] - scaler.mean_) / scaler.scale_
        x_pca = pca_denoise.transform(x_noisy_scaled.reshape(1, -1))
        x_reconstructed = pca_denoise.inverse_transform(x_pca)
        x_reconstructed = x_reconstructed * scaler.scale_ + scaler.mean_
        
        axes[i, j+2].imshow(x_reconstructed.reshape(8, 8), cmap='gray')
        if i == 0:
            var_explained = pca_denoise.explained_variance_ratio_.sum() * 100
            axes[i, j+2].set_title(f'n={n_comp}\n({var_explained:.0f}%)', fontsize=10)
        axes[i, j+2].axis('off')

# Etichette righe
for i, idx in enumerate(indices):
    axes[i, 0].set_ylabel(f'Cifra {y_digits[idx]}', fontsize=10)

plt.suptitle('PCA per Riduzione Rumore: Originale → Rumoroso → Ricostruzione', fontsize=13)
plt.tight_layout()
plt.show()

print("""
📊 OSSERVAZIONI:

1. Con poche componenti (5-10): 
   - Rimuove molto rumore
   - Ma perde anche dettagli

2. Con più componenti (20-40):
   - Mantiene più dettagli
   - Ma rimuove meno rumore

3. Trade-off:
   - Poche PC → più smoothing, meno dettaglio
   - Molte PC → più dettaglio, meno denoising

🎯 PCA filtra il rumore perché il rumore è "random" 
   e non viene catturato dalle componenti principali!
""")

---

## 📝 4. Esercizi

### 📝 Esercizio 24.1 — PCA su Dati di Vini

**Consegna:** Usa PCA per visualizzare e analizzare il dataset Wine.

**Richieste:**
1. Carica il dataset Wine (sklearn.datasets.load_wine)
2. Scala i dati e applica PCA
3. Determina quante componenti servono per l'80% e 95% della varianza
4. Visualizza i dati in 2D colorati per tipo di vino
5. Interpreta le prime 2 PC analizzando i loadings

In [ ]:
# ============================================
# ESERCIZIO 24.1 — SOLUZIONE
# ============================================
from sklearn.datasets import load_wine

print("="*70)
print("ESERCIZIO 24.1 — PCA su Dataset Wine")
print("="*70)

# ============================================
# PASSO 1: Carica e prepara i dati
# ============================================
wine = load_wine()
X_wine = wine.data
y_wine = wine.target
feature_names_wine = wine.feature_names

print(f"\n📊 Dataset Wine:")
print(f"   - Campioni: {X_wine.shape[0]}")
print(f"   - Feature: {X_wine.shape[1]}")
print(f"   - Classi: {wine.target_names}")

# Scaling
scaler = StandardScaler()
X_wine_scaled = scaler.fit_transform(X_wine)

# ============================================
# PASSO 2: PCA esplorativo
# ============================================
pca_wine = PCA()
pca_wine.fit(X_wine_scaled)

cumsum_var = np.cumsum(pca_wine.explained_variance_ratio_)
n_80 = np.argmax(cumsum_var >= 0.80) + 1
n_95 = np.argmax(cumsum_var >= 0.95) + 1

print(f"\n📈 Componenti necessarie:")
print(f"   - Per 80% varianza: {n_80} componenti")
print(f"   - Per 95% varianza: {n_95} componenti")

# ============================================
# PASSO 3: Visualizzazione
# ============================================
pca_2d = PCA(n_components=2)
X_wine_2d = pca_2d.fit_transform(X_wine_scaled)

fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Scree plot
axes[0].bar(range(1, 14), pca_wine.explained_variance_ratio_, alpha=0.7, color='steelblue')
axes[0].plot(range(1, 14), cumsum_var, 'ro-', label='Cumulativa')
axes[0].axhline(y=0.80, color='orange', linestyle='--', alpha=0.7, label='80%')
axes[0].axhline(y=0.95, color='red', linestyle='--', alpha=0.7, label='95%')
axes[0].set_xlabel('Componente', fontsize=11)
axes[0].set_ylabel('Varianza Spiegata', fontsize=11)
axes[0].set_title('Scree Plot', fontsize=12)
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# 2D scatter
colors = ['red', 'green', 'blue']
for i, name in enumerate(wine.target_names):
    mask = y_wine == i
    axes[1].scatter(X_wine_2d[mask, 0], X_wine_2d[mask, 1], 
                    c=colors[i], label=name, s=50, alpha=0.7, edgecolors='black')
axes[1].set_xlabel(f'PC1 ({pca_2d.explained_variance_ratio_[0]*100:.1f}%)')
axes[1].set_ylabel(f'PC2 ({pca_2d.explained_variance_ratio_[1]*100:.1f}%)')
axes[1].set_title('Wine Dataset in 2D', fontsize=12)
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# Loadings heatmap
loadings = pca_2d.components_
im = axes[2].barh(range(len(feature_names_wine)), loadings[0], alpha=0.7, label='PC1')
axes[2].barh(range(len(feature_names_wine)), loadings[1], alpha=0.7, label='PC2', left=loadings[0])
axes[2].set_yticks(range(len(feature_names_wine)))
axes[2].set_yticklabels(feature_names_wine, fontsize=8)
axes[2].set_xlabel('Loading', fontsize=11)
axes[2].set_title('Loadings PC1 e PC2', fontsize=12)
axes[2].legend()
axes[2].axvline(x=0, color='black', linestyle='-', linewidth=0.5)
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# ============================================
# PASSO 4: Interpretazione
# ============================================
print("\n" + "="*70)
print("PASSO 4: Interpretazione Loadings")
print("="*70)

# Top 3 feature per PC1 e PC2
df_loadings = pd.DataFrame({
    'Feature': feature_names_wine,
    'PC1': loadings[0],
    'PC2': loadings[1]
})

print("\n📊 Top 3 feature per PC1 (ordine assoluto):")
top_pc1 = df_loadings.reindex(df_loadings['PC1'].abs().sort_values(ascending=False).index)
print(top_pc1.head(3).to_string(index=False))

print("\n📊 Top 3 feature per PC2 (ordine assoluto):")
top_pc2 = df_loadings.reindex(df_loadings['PC2'].abs().sort_values(ascending=False).index)
print(top_pc2.head(3).to_string(index=False))

print("""
📌 INTERPRETAZIONE:
   - PC1: Cattura principalmente la "intensità" del vino 
     (flavanoids, proline, color_intensity)
   - PC2: Distingue per contenuto "minerale/acido"
     (alcalinity_of_ash, nonflavanoid_phenols)
""")

---

## 🏋️ Esercizio 24.2 — PCA come Preprocessing per Classificazione

**Obiettivo:** Usare PCA per ridurre le dimensioni prima di un classificatore.

**Consegna:**
1. Usa il dataset **Digits** (load_digits)
2. Applica PCA conservando il **90%** della varianza
3. Addestra un **LogisticRegression** sui dati originali e sui dati ridotti
4. Confronta:
   - Accuracy su test set
   - Tempo di training
   - Numero di feature usate

**Domanda:** La riduzione di dimensionalità migliora o peggiora le prestazioni?

In [ ]:
# ============================================
# ESERCIZIO 24.2 — SOLUZIONE
# ============================================
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import time

print("="*70)
print("ESERCIZIO 24.2 — PCA come Preprocessing per Classificazione")
print("="*70)

# ============================================
# PASSO 1: Prepara i dati
# ============================================
X_train, X_test, y_train, y_test = train_test_split(
    X_digits, y_digits, test_size=0.2, random_state=42
)

# Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"\n📊 Dataset:")
print(f"   - Training: {X_train_scaled.shape}")
print(f"   - Test: {X_test_scaled.shape}")

# ============================================
# PASSO 2: Training SENZA PCA
# ============================================
print("\n" + "="*50)
print("SCENARIO 1: Senza PCA (64 feature)")
print("="*50)

start = time.time()
lr_no_pca = LogisticRegression(max_iter=5000, random_state=42)
lr_no_pca.fit(X_train_scaled, y_train)
time_no_pca = time.time() - start

acc_no_pca = lr_no_pca.score(X_test_scaled, y_test)
print(f"   ⏱️  Tempo training: {time_no_pca:.4f}s")
print(f"   📊 Accuracy: {acc_no_pca*100:.2f}%")

# ============================================
# PASSO 3: Training CON PCA (90% varianza)
# ============================================
print("\n" + "="*50)
print("SCENARIO 2: Con PCA (90% varianza)")
print("="*50)

pca_90 = PCA(n_components=0.90)
X_train_pca = pca_90.fit_transform(X_train_scaled)
X_test_pca = pca_90.transform(X_test_scaled)

n_components = pca_90.n_components_
print(f"   📐 Componenti selezionate: {n_components}")

start = time.time()
lr_pca = LogisticRegression(max_iter=5000, random_state=42)
lr_pca.fit(X_train_pca, y_train)
time_pca = time.time() - start

acc_pca = lr_pca.score(X_test_pca, y_test)
print(f"   ⏱️  Tempo training: {time_pca:.4f}s")
print(f"   📊 Accuracy: {acc_pca*100:.2f}%")

# ============================================
# PASSO 4: Confronto
# ============================================
print("\n" + "="*70)
print("CONFRONTO FINALE")
print("="*70)

comparison = pd.DataFrame({
    'Metrica': ['Feature', 'Tempo (s)', 'Accuracy (%)'],
    'Senza PCA': [64, f"{time_no_pca:.4f}", f"{acc_no_pca*100:.2f}"],
    'Con PCA': [n_components, f"{time_pca:.4f}", f"{acc_pca*100:.2f}"]
})
print(comparison.to_string(index=False))

speedup = time_no_pca / time_pca if time_pca > 0 else 1
feature_reduction = (1 - n_components/64) * 100
acc_diff = (acc_pca - acc_no_pca) * 100

print(f"""
📌 CONCLUSIONI:
   - Riduzione feature: {feature_reduction:.1f}% (da 64 a {n_components})
   - Speedup training: {speedup:.2f}x
   - Differenza accuracy: {acc_diff:+.2f}%
   
💡 PCA riduce drasticamente le dimensioni mantenendo 
   prestazioni quasi identiche → ottimo per dataset grandi!
""")

---

## 🏋️ Esercizio 24.3 — Confronto Spazio Originale vs PCA

**Obiettivo:** Visualizzare l'effetto della riduzione dimensionale sulla separabilità delle classi.

**Consegna:**
1. Usa il dataset **Iris**
2. Crea una visualizzazione con:
   - **Pannello 1**: Scatter plot di 2 feature ORIGINALI (sepal length vs sepal width)
   - **Pannello 2**: Scatter plot PC1 vs PC2
3. Calcola e confronta la **distanza media tra i centroidi** delle classi nei due spazi
4. Quale rappresentazione separa meglio le classi?

In [ ]:
# ============================================
# ESERCIZIO 24.3 — SOLUZIONE
# ============================================
from scipy.spatial.distance import cdist

print("="*70)
print("ESERCIZIO 24.3 — Confronto Spazio Originale vs PCA")
print("="*70)

# ============================================
# PASSO 1: Prepara i dati
# ============================================
iris = load_iris()
X_iris = iris.data
y_iris = iris.target
target_names = iris.target_names

# Spazio originale: solo 2 feature (sepal)
X_orig = X_iris[:, :2]  # sepal length, sepal width

# Spazio PCA
scaler = StandardScaler()
X_iris_scaled = scaler.fit_transform(X_iris)
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_iris_scaled)

print(f"📊 Varianza spiegata da PC1+PC2: {pca.explained_variance_ratio_.sum()*100:.1f}%")

# ============================================
# PASSO 2: Visualizzazione
# ============================================
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

colors = ['red', 'green', 'blue']
markers = ['o', 's', '^']

# Pannello 1: Spazio originale
for i, name in enumerate(target_names):
    mask = y_iris == i
    axes[0].scatter(X_orig[mask, 0], X_orig[mask, 1], 
                    c=colors[i], marker=markers[i], s=80,
                    label=name, alpha=0.7, edgecolors='black')
axes[0].set_xlabel('Sepal Length', fontsize=12)
axes[0].set_ylabel('Sepal Width', fontsize=12)
axes[0].set_title('Spazio Originale (2 feature)', fontsize=14)
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Pannello 2: Spazio PCA
for i, name in enumerate(target_names):
    mask = y_iris == i
    axes[1].scatter(X_pca[mask, 0], X_pca[mask, 1], 
                    c=colors[i], marker=markers[i], s=80,
                    label=name, alpha=0.7, edgecolors='black')
axes[1].set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]*100:.1f}%)', fontsize=12)
axes[1].set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]*100:.1f}%)', fontsize=12)
axes[1].set_title('Spazio PCA (2 componenti)', fontsize=14)
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# ============================================
# PASSO 3: Calcolo distanze tra centroidi
# ============================================
print("\n" + "="*70)
print("ANALISI SEPARABILITÀ — Distanza tra Centroidi")
print("="*70)

def calcola_distanza_media_centroidi(X, y):
    """Calcola la distanza media tra tutti i centroidi delle classi."""
    centroids = []
    for c in np.unique(y):
        centroids.append(X[y == c].mean(axis=0))
    centroids = np.array(centroids)
    
    # Matrice delle distanze tra centroidi
    dist_matrix = cdist(centroids, centroids)
    
    # Media delle distanze (esclusa diagonale)
    n = len(centroids)
    total_dist = 0
    count = 0
    for i in range(n):
        for j in range(i+1, n):
            total_dist += dist_matrix[i, j]
            count += 1
    return total_dist / count, centroids

dist_orig, centroids_orig = calcola_distanza_media_centroidi(X_orig, y_iris)
dist_pca, centroids_pca = calcola_distanza_media_centroidi(X_pca, y_iris)

print(f"\n📐 Distanza media tra centroidi:")
print(f"   - Spazio originale: {dist_orig:.3f}")
print(f"   - Spazio PCA: {dist_pca:.3f}")

improvement = (dist_pca - dist_orig) / dist_orig * 100

print(f"""
📌 CONCLUSIONI:
   - PCA aumenta la distanza tra centroidi del {improvement:.1f}%
   - Questo significa che le classi sono PIÙ SEPARATE in PCA
   - PCA massimizza la varianza → massimizza la separazione!
   
💡 Anche se PCA è unsupervised (non usa le label), 
   spesso migliora la separabilità perché le classi 
   tendono a differire lungo le direzioni di massima varianza.
""")

---

# 🎯 5. Conclusione — Cosa Portarsi a Casa

## ✅ Concetti Chiave Appresi

| Concetto | Descrizione |
|----------|-------------|
| **PCA** | Trova le direzioni di massima varianza nei dati |
| **Componenti Principali** | Nuove feature ordinate per importanza |
| **Varianza Spiegata** | Quanto "informazione" cattura ogni PC |
| **Loadings** | Pesi delle feature originali in ogni PC |

## 🚨 Errori Comuni da Evitare

1. **Non scalare i dati** → Feature con range diversi dominano PCA
2. **Tenere troppe/poche componenti** → Usa scree plot + soglia 80-95%
3. **Ignorare i loadings** → Perdi interpretabilità del modello
4. **Applicare PCA a dati categorici** → PCA è per dati numerici continui

## 💡 Quando Usare PCA

```
✅ USA PCA:
   - Molte feature correlate
   - Visualizzazione di dati high-dimensional
   - Preprocessing per velocizzare training
   - Rimuovere rumore

❌ NON USARE PCA:
   - Poche feature indipendenti
   - Serve interpretabilità diretta
   - Dati categorici o sparsi
```

## 🔗 Ponte verso la Prossima Lezione

Nella **Lezione 25** combineremo **PCA + Clustering**:
- PCA per ridurre le dimensioni
- K-Means o DBSCAN sui dati ridotti
- Visualizzazione dei cluster in 2D

Questo workflow è molto comune nel mondo reale!

---

# 📚 6. Bignami — Scheda di Riferimento Rapido

## 📖 Definizioni Essenziali

| Termine | Definizione |
|---------|-------------|
| **Componente Principale (PC)** | Direzione di massima varianza, combinazione lineare delle feature originali |
| **Varianza Spiegata** | Frazione della varianza totale catturata da una PC (valori 0-1) |
| **Loadings** | Coefficienti che indicano il contributo di ogni feature a una PC |
| **Scree Plot** | Grafico per visualizzare la varianza spiegata per componente |
| **Curse of Dimensionality** | Problemi che emergono con molte feature: sparsità, overfitting, calcolo |

## 📐 Regole per Scegliere n_components

| Metodo | Regola |
|--------|--------|
| **Soglia Varianza** | Mantieni 80-95% della varianza cumulativa |
| **Kaiser** | Mantieni PC con eigenvalue > 1 (dopo StandardScaler) |
| **Gomito** | Trova il "gomito" nello scree plot |

## 🔧 Template di Codice

### PCA Base
```python
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# 1. Scala (OBBLIGATORIO!)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 2. PCA con n fisso
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

# 3. PCA con varianza target
pca = PCA(n_components=0.95)  # 95% varianza
X_pca = pca.fit_transform(X_scaled)
```

### Scree Plot
```python
pca_full = PCA()
pca_full.fit(X_scaled)

cumsum = np.cumsum(pca_full.explained_variance_ratio_)
plt.bar(range(1, len(cumsum)+1), pca_full.explained_variance_ratio_)
plt.plot(range(1, len(cumsum)+1), cumsum, 'ro-')
plt.axhline(y=0.95, color='r', linestyle='--')
```

### Loadings Heatmap
```python
loadings = pd.DataFrame(
    pca.components_.T,
    columns=[f'PC{i+1}' for i in range(pca.n_components_)],
    index=feature_names
)
sns.heatmap(loadings, annot=True, cmap='RdBu_r', center=0)
```

## ✅ Checklist Pre-PCA

- [ ] Dati numerici continui?
- [ ] Feature scalate con StandardScaler?
- [ ] Feature correlate (senso applicare PCA)?
- [ ] Definita soglia varianza target?
- [ ] Piano per interpretare i loadings?

---

**🎉 Fine Lezione 24 — PCA Masterclass!**